In [10]:
# %matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
import matplotlib
import matplotlib.pyplot as plt

import glob
import itertools
import os

from run import Run
from runs_db import db as runs_db

In [12]:
log_path = '/home/fiit/logs'
runs = []

for server in runs_db:
    paths = glob.glob(os.path.join(log_path, server, '*'))
    paths = iter(sorted(paths))

    try:
        for (number, type_, code) in runs_db[server]:
            for _ in range(number):
                try:
                    path = next(paths)
                    runs.append(Run(path, type_, code))
                except KeyError:
                    print(path)
    except StopIteration:
        pass


In [13]:
tasks = ['dep', 'lmo', 'ner', 'pos']
langs = ['cs', 'de', 'en', 'es']

task_metr = {
    'dep': 'las',
    'lmo': 'perplexity',
    'ner': 'chunk_f1',
    'pos': 'acc'
}

task_max = {
    'dep': True,
    'lmo': False,
    'ner': True,
    'pos': True
}

def draw_graphs(metric_func, tasks, langs, role, run_codes=None, run_types=None, focused=False):
    fig, axes = plt.subplots(len(tasks), len(langs), figsize=(5*len(langs), 4*len(tasks)), squeeze=False)

    relevant_runs = ['vanilla']

    for task, lang in itertools.product(tasks, langs):
        for run in runs:
            if (
                (not focused or run.config['focus_on'] == f'{task}-{lang}') and
                (run_codes is None or run.code in run_codes) and
                (run_types is None or run.type in run_types)
            ):

                history = run.history(
                    metric=metric_func(task),
                    task=task,
                    language=lang,
                    role=role)
                axes[tasks.index(task), langs.index(lang)].plot(list(history), label=f'{run.code}-{run.type}')
                #axes[tasks.index(task), langs.index(lang)].plot(list(history), label=run.type)

    for ax, col in zip(axes[0], langs):
        ax.set_title(col)

    for ax, row in zip(axes[:, 0], tasks):
        ax.set_ylabel(row, rotation=0, size='large')
        
    for ax_row in axes:
        for ax in ax_row:
            ax.legend()

    plt.show()
    
def find_runs(run_code=None, run_type=None, contains=None, **config):
    
    if contains is None:
        contains = []
    
    return (run
           for run
           in runs
           if  (run_code is None or run_code == run.code) and
               (run_type is None or run_type == run.type) and
               all(run.contains(*task_lang) for task_lang in contains) and
               all(run.config[key] == value for key, value in config.items()))
    
    
def print_results(runs, tasks, langs, metric_func=None, metric_max_func=None, focused=True):
        
    if metric_func is None:
        metric_func = lambda task: task_metr[task]
        
    if metric_max_func is None:
        metric_max_func = lambda task: task_max[task]
        
    output = []

    for run in runs:
        for task, lang in itertools.product(tasks, langs):
            if not focused or run.config['focus_on'] == f'{task}-{lang}':
                res, epoch = run.metric_eval(
                    metric=metric_func(task),
                    max_=metric_max_func(task),
                    task=task,
                    language=lang)
                if res <= 1.01:
                    res *= 100
                #print(task, lang, res, epoch)
                print(f'{res:.2f}', end=' ')
                output.append((res, epoch, run))

    return output

In [17]:
def tempo(code, type_='all'):    
    print_results(
        runs=find_runs(
            run_type=type_,
            run_code=code
        ),
        tasks=tasks,
        langs=langs,
    )
    print()
   
tempo('zero-shot', 'ml-3')
tempo('zero-shot', 'rel')
tempo('zero-shot')
print()
tempo('zero-shot')
tempo('zero-shot-task')
tempo('zero-shot-lang')
tempo('zero-shot-task-lang-no-global')
tempo('zero-shot-task-lang')
print()
tempo('zero-shot-task-lang')
tempo('zero-shot-task-lang-ortho-10')
tempo('zero-shot-task-lang-ortho-25')
tempo('zero-shot-task-lang-ortho-50')
tempo('zero-shot-task-lang-ortho-100')
tempo('zero-shot-task-lang-ortho-200')
print()
tempo('zero-shot-task-lang-no-global')
tempo('zero-shot-task-lang-no-global-ortho-50')
tempo('zero-shot-task-lang-no-global-ortho-100')
tempo('zero-shot-task-lang-no-global-ortho-200')
print()
tempo('zero-shot')
tempo('zero-shot-task-emb')
tempo('zero-shot-lang-emb')
tempo('zero-shot-embs')
tempo('zero-shot-embs-400')
tempo('zero-shot-task-lang-both-embs')
print()
tempo('zero-shot')
tempo('zero-shot-task-emb')
tempo('zero-shot-lang-emb')
tempo('zero-shot-embs')
tempo('zero-shot-embs-400')
tempo('zero-shot-task-lang-both-embs')
print()
tempo('zero-shot', 'ml-3')
tempo('zero-shot')
tempo('zero-shot-embs')
tempo('zero-shot-task-lang')
print()
tempo('zero-shot-rotated', 'ml-3')
tempo('zero-shot-rotated')
tempo('zero-shot-embs-rotated')
tempo('zero-shot-task-lang-rotated')
print()
tempo('zero-shot-char-level', 'ml-3')
tempo('zero-shot-char-level')
tempo('zero-shot-embs-char-level')
tempo('zero-shot-task-lang-char-level')







# draw_graphs(lambda t: task_metr[t], tasks, langs, 'test', run_types=['all'], run_codes=[
#     'zero-shot-task-emb',
#     'zero-shot-lang-emb', 
#     'zero-shot-embs',
#     'zero-shot',
# ], focused=True)
# draw_graphs(lambda t: task_metr[t], tasks, langs, 'test', run_types=['stsl'], run_codes=[
#     'normal-training',
# ])

#draw_graphs(lambda t: 'ortho', tasks, langs, 'test', run_codes=[code])
#print_results(find_runs(run_code=code), [task], [lang], lambda t: task_metr[t], lambda t: task_max[t])

[('deepnet5', 288), ('deepnet2070', 192)]
30.47 44.76 47.75 32.02 40.17 55.91 47.83 53.27 65.78 77.66 75.05 57.72 
33.39 43.37 42.19 30.59 45.61 60.49 50.18 56.22 76.32 78.95 80.87 73.21 
43.61 54.03 44.38 42.97 49.11 62.71 53.12 61.54 83.12 85.02 83.40 84.81 

43.61 54.03 44.38 42.97 49.11 62.71 53.12 61.54 83.12 85.02 83.40 84.81 
49.54 57.50 53.45 52.26 46.78 60.81 52.86 60.94 86.73 86.62 82.47 89.78 
43.55 52.70 54.30 52.26 45.55 64.55 54.03 61.14 74.89 82.79 78.18 76.49 
51.29 59.22 58.05 59.85 48.45 64.84 51.38 60.87 83.85 86.84 82.43 87.46 
52.35 59.90 58.58 60.65 49.63 64.72 53.51 62.35 85.55 87.19 82.35 88.71 

52.35 59.90 58.58 60.65 49.63 64.72 53.51 62.35 85.55 87.19 82.35 88.71 
52.62 60.34 59.05 61.11 48.68 63.43 53.70 62.73 84.94 87.71 82.69 89.00 
52.94 58.65 59.16 61.52 47.62 62.48 54.40 61.66 84.88 85.45 81.89 86.16 
54.68 60.50 58.07 60.84 49.29 64.10 54.31 62.18 84.75 87.78 82.86 87.10 
53.18 59.66 57.12 61.60 48.69 64.19 54.55 60.34 85.26 87.54 81.86 88.00 
50.74 5